In [2]:
import torch
import numpy as np

#### 1. 使用 tensor 初始化一个 1 × 3 的矩阵M 和一个2 × 1 的矩阵N，对两矩阵进行减法操作（要求实现三种不同的形式），给出结果并分析三种方式的不同（如果出现报错，分析报错的原因），同时需要指出在计算过程中发生了什么

In [3]:

li_m = [[1, 2, 3]] # 1 x 3
li_n = [[1], [2]] # 2 x 1
M = torch.tensor(li_m)
N = torch.tensor(li_n)
print(M-N)

tensor([[ 0,  1,  2],
        [-1,  0,  1]])


In [4]:

print(torch.sub(M, N))

tensor([[ 0,  1,  2],
        [-1,  0,  1]])


In [5]:
print(M.sub_(N))

RuntimeError: output with shape [1, 3] doesn't match the broadcast shape [2, 3]

## 基本操作实验2
1) 利用 𝐓𝐞𝐧𝐬𝐨𝐫 创建两个大小分别 𝟑×𝟐 和 𝟒×𝟐 的随机数矩阵 𝑷 和 𝑸 ，要求服从均值为0，标准差0.01为的正态分布 \
2) 对第二步得到的矩阵 𝑸 进行形状变换得到 𝑸 的转置 $𝑸^𝑻$  \
3) 对上述得到的矩阵 𝑷 和矩阵 $𝑸^𝑻$ 求内积

In [ ]:
P = torch.normal(0, 0.01, size=(3, 2))
Q = torch.normal(0, 0.01, size=(4, 2))
print('P:{}\nQ:{}'.format(P, Q))

P:tensor([[ 0.0127, -0.0046],
        [-0.0085,  0.0122],
        [-0.0041, -0.0023]])
Q:tensor([[ 0.0069, -0.0133],
        [-0.0095, -0.0051],
        [ 0.0098,  0.0059],
        [ 0.0149,  0.0137]])


In [ ]:
Q_T = Q.T # size = 2 x 4
Q_T 

tensor([[ 0.0069, -0.0095,  0.0098,  0.0149],
        [-0.0133, -0.0051,  0.0059,  0.0137]])

In [ ]:
# 矩阵运算P和Q.t
torch.mm(P, Q_T)

tensor([[ 1.4737e-04, -9.6249e-05,  9.7251e-05,  1.2602e-04],
        [-2.2110e-04,  1.7617e-05, -1.1467e-05,  4.0608e-05],
        [ 2.3747e-06,  5.0479e-05, -5.3604e-05, -9.2226e-05]])

## 基本操作实验3
给定公式$𝑦_3=𝑦_1+𝑦_2=𝑥^2+𝑥^3$，且 $𝑥=1$。利用学习所得到的Tensor的相关知识，求$𝑦_3$对的梯度𝑥，即$𝑑𝑦_3$/$𝑑𝑥$。要求在计算过程中，在计算 $𝑥^3$ 时中断梯度的追踪，观察结果并进行原因分析

$x^3$的梯度是2而不是5的原因：\
由于 $y_2$的定义是被torch.no_grad():包裹的，所以与$y_2$有关的梯度是不会回传的，只有与$y_1$有关的梯度才会回传，即$x^2$对 $x$的梯度。\
而y2.requires_grad=False，所以不能调用 y2.backward()，会报错

In [7]:
# 根据题目给的已知进行初始化
x = torch.ones(1, requires_grad=True)
y1 = torch.pow(x, 2)
# 中断x^3的梯度追踪
with torch.no_grad():
    y2 = torch.pow(x, 3)
y3 = y1 + y2
x.requires_grad, y1.requires_grad, y2.requires_grad, y3.requires_grad

(True, True, False, True)

In [8]:
x.requires_grad, y1.requires_grad, y2.requires_grad, y3.requires_grad

(True, True, False, True)

In [ ]:
# o1来记录y1
o1 = y1.sum()
o1.backward(retain_graph=True)
x.grad

tensor([4.])

In [ ]:
# o3来记录y3
o3 = y3.sum()
x.grad.zero_()  # 如果没有清零会有叠加
o3.backward(retain_graph=True)
x.grad

tensor([2.])

In [ ]:
# 调用禁用的变量y2
o2 = y2.sum()
o2.backward()

# 出错了

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn